In [10]:
from src.signal_generator import SignalGenerator
from src.metrics import Metric
from src.data_type import Measurement
import matplotlib.pyplot as plt
from src.data_compressor.other import CompressNTHS
from src.data_compressor.other import CompressMinMax
from src.data_compressor.other import CompressPWP
from src.data_compressor.pip import CompressPIP_ED
from src.data_compressor.pip import CompressPIP_PD
from src.data_compressor.pip import CompressPIP_VD
from src.data_compressor.paa import CompressPAA
from src.data_compressor.paa import CompressPAAVI
from src.data_compressor.paa import CompressByChunk
from src.data_compressor.pla import CompressAPCADFT
from src.data_compressor.pla import CompressAPCAFFT
from src.data_compressor.pla import CompressSTC
from src.data_compressor.pla import CompressHigherDeriveration
from src.data_type import Measurement
from src.data_compressor.compressor import Compressor
from typing import List, Dict
from collections import defaultdict as dict

from src.method_selector import AlgorythmSelector

In [11]:
import matplotlib.pyplot as plt
from typing import List
def vizualize(original_data, compressed_data, show_compressed: bool = True, show_interpolation: bool = True):
  x_original = [measurement.timestamp for measurement in original_data]
  y_original = [measurement.value for measurement in original_data]
  plt.figure(dpi=150)
  if show_interpolation:
      plt.plot(x_original, y_original, 'b')
  plt.plot(x_original, y_original, 'bo')
  if show_compressed:
    x_compressed = [measurement.timestamp for measurement in compressed_data]
    y_compressed = [measurement.value for measurement in compressed_data]
    if show_interpolation:
      plt.plot(x_compressed, y_compressed, 'r')
    plt.plot(x_compressed, y_compressed, 'ro')
  plt.grid()
  plt.show()

# vizualize(data, self.metrics_containter._interpolate_data(data, compressed_data))

In [12]:
def to_measurements(signal_generator: SignalGenerator) -> List[Measurement]:
  return [Measurement(measurement, index * 100) for index, measurement in enumerate(signal_generator.data)]

def strip_data(data: List[Measurement]) -> List[float]:
  return [item.value for item in data]

In [13]:
dataset = []
signal_generators = [
  SignalGenerator(0, 100).with_peaks(3).with_peaks(3, direction=-1).sin(0.2, 0.2),
  SignalGenerator(0, 100).with_peaks(3).with_peaks(3, direction=-1).noise(),
  SignalGenerator(0, 100).with_peaks(3).noise(),
  SignalGenerator(0, 100).with_peaks(3),
  SignalGenerator(0, 100).square_vawe(),
  SignalGenerator(0, 100).square_vawe().with_peaks(5),
  SignalGenerator(0, 100).square_vawe().with_peaks(5).sin(),
  SignalGenerator(0, 100).square_vawe().with_peaks(5).sin().noise(),
]

for signal_generator in signal_generators:
  measurements = to_measurements(signal_generator)
  best_method_name = AlgorythmSelector().get_best(measurements)[0].__class__.__name__
  dataset.append([strip_data(measurements), best_method_name]) # tu powinny być metryki na wejściu

In [14]:
from sklearn import tree
from fogml.generators import GeneratorFactory

dataset_size = len(dataset)

X_train = [data for data, label in dataset]
y_train = [label for data, label in dataset]

X_test = [data for data, label in dataset[:2]]
y_test = [label for data, label in dataset[:2]]

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

score = clf.score(X_test, y_test)
print(f'Score: {score}')

factory = GeneratorFactory()
generator = factory.get_generator(clf)
generator.generate(fname='output.c')

Score: 1.0
